In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('sales.csv')
df.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Street,SaleCondition
0,208500,1710,548,7,Pave,Normal
1,181500,1262,460,6,Pave,Normal
2,223500,1786,608,7,Pave,Normal
3,140000,1717,642,7,Pave,Abnorml
4,250000,2198,836,8,Pave,Normal


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SalePrice      1460 non-null   int64 
 1   GrLivArea      1460 non-null   int64 
 2   GarageArea     1460 non-null   int64 
 3   OverallQual    1460 non-null   int64 
 4   Street         1460 non-null   object
 5   SaleCondition  1460 non-null   object
dtypes: int64(4), object(2)
memory usage: 68.6+ KB


Данные по продаже квартир содержат информаю по 6 характеристикам, всего 1460 записей. Пропусков в данных нет.
4 характеристики (переменные): имеют числовое целочисленное значение, тип int64
2 характеристики  - категориальные, имеют тип object.

In [6]:
df.describe()

,SalePrice,GrLivArea,GarageArea,OverallQual
count,1460.000000,1460.000000,1460.000000,1460.000000
mean,180921.195890,1515.463699,472.980137,6.099315
std,79442.502883,525.480383,213.804841,1.382997
min,34900.000000,334.000000,0.000000,1.000000
25%,129975.000000,1129.500000,334.500000,5.000000
50%,163000.000000,1464.000000,480.000000,6.000000
75%,214000.000000,1776.750000,576.000000,7.000000
max,755000.000000,5642.000000,1418.000000,10.000000


In [7]:
df.describe(include='object')

,Street,SaleCondition
count,1460,1460
unique,2,6
top,Pave,Normal
freq,1454,1198


In [8]:
#Преобразуем колонку Street через One-hot encoding
one_hot = pd.get_dummies(df["Street"], drop_first=True).astype('int')
df = pd.concat((df.drop(["Street"], axis=1), one_hot), axis=1)
df.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,SaleCondition,Pave
0,208500,1710,548,7,Normal,1
1,181500,1262,460,6,Normal,1
2,223500,1786,608,7,Normal,1
3,140000,1717,642,7,Abnorml,1
4,250000,2198,836,8,Normal,1


In [9]:
#Преобразуем колонку SaleCondition через One-hot encoding
one_hot = pd.get_dummies(df["SaleCondition"], drop_first=True).astype('int')
df = pd.concat((df.drop(["SaleCondition"], axis=1), one_hot), axis=1)
df.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Pave,AdjLand,Alloca,Family,Normal,Partial
0,208500,1710,548,7,1,0,0,0,1,0
1,181500,1262,460,6,1,0,0,0,1,0
2,223500,1786,608,7,1,0,0,0,1,0
3,140000,1717,642,7,1,0,0,0,0,0
4,250000,2198,836,8,1,0,0,0,1,0


In [10]:
from sklearn.model_selection import train_test_split

X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
print('Linear regression, Train')
print(f'MSE: {mean_squared_error(y_train, y_train_pred)},\
\nMAE: {mean_absolute_error( y_train, y_train_pred)}')

Linear regression, Train
MSE: 1550129089.6633768,
MAE: 26178.7058330456


In [13]:
y_test_pred = lr.predict(X_test)
print('Linear regression, Test')
print(f'MSE: {mean_squared_error( y_test, y_test_pred)},\
\nMAE: {mean_absolute_error( y_test, y_test_pred)}')

Linear regression, Test
MSE: 1764817019.807445,
MAE: 27253.180178728442


Качество модели линейной регрессии получилось среднее, MAE досстаточно высок, с учетом того, что цены на квартире в среднем равны 180921 со стандартным отклонением 79443. При этом диапазон цен от 34900 до 755000. 
Модель линейной регрессии не переобучилась, ошибки на тренировочнйо и тестовой выборках соезмеримы.